<div style="width:100%; text-align:center; margin: 40px 0;">
    <h1 style="color: #003366; font-size: 40px; font-weight:bold; text-shadow: 2px 2px 5px grey;">
        Énergie Électrique en France
    </h1>
    <hr style="width: 50%; height: 3px; background-color: #003366; border: none;">
    <p style="font-style:italic; font-size:18px; color: #555;">
        Une exploration approfondie de l'énergie électrique en France
    </p>
</div>


<h2 style="padding:16px; color:#FFF; background:#07D">Imports</h2>

In [34]:
# Data
URL = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/"

# Imports
import requests
import sqlite3
import time
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, VBox, HBox, DatePicker, Output, Dropdown
from IPython.display import display, clear_output

<h2 style="padding:16px; color:#FFF; background:#07D">I. Présentation des données</h2>

éCO2mix est un outil créé par RTE France afin de mieux connaître en temps réel l'électricité en France. RTE France met à disposition gratuitement, via plusieurs API, les base de données utilisée par éCO2mix qui fournissent les données de 2013 à aujourd'hui. Les informations fournient par les API sont les suivantes :

- Consommation d’électricité en France métropolitaine (hors Corse) et prévisions calculées la veille et en début d’après-midi le jour J ;
- Production d’électricité française par filière en valeur et en pourcentage ;
- Échanges commerciaux d’électricité avec les 6 pays voisins ;
- Émissions de CO2 que génère la production d’électricité ;
- Données mensuelles à l’échelle régionale : consommation, production et flux interrégionaux.

Les données sont séparées en 4 bases de données :

- Données éCO2mix nationales temps réel
- Données éCO2mix nationales consolidées et définitives
- Données éCO2mix régionales temps réel
- Données éCO2mix régionales consolidées et définitives

<h2 style="padding:16px; color:#FFF; background:#07D">II. Objectif de l'étude</h2>

Cette étude a pour but l'analyse de la production, de la consommation et des échanges d'électricité en France depuis 2013. L'objectif est de déterminer les parts des différentes filières dans la production d'électricité et de voir leur évolution au cours du temps, de comprendre les tendances dans la consommation d'électricité sur plusieurs échelles de temps et d'identifier la tendance commerciale de la France avec ses pays voisins sur le marché de l'électricité. Enfin, cette étude nous permettra d'étudier l'impact de la production d'électricité sur les émissions de CO2 en France.

<h2 style="padding:16px; color:#FFF; background:#07D">III. Récupération des données</h2>

In [2]:
def fetch_data_by_date(data:str, start:int, rows:int, date:str):
    """Fetch data from a dataset by date and offset
    
    Parameters
    ----------
    data : str
        Name of the dataset.
    start : int
        Offset.
    rows : int
        Number of rows.
    date : str
        Date of the data.

    Returns
    -------
    dict
        Dictionary containing the data. 
    """
    url = f"{URL}{data}" + "/records"
    params = {
        "offset" : start,
        "rows": rows,
        "where": f"date='{date}'"
    }
    response = requests.get(url, params=params)
    data=[{}]
    if response.status_code == 200:
        data = response.json()
        return data.get('results', {})
    else:
        print(f"Échec de la requête: {response.status_code}")
        print(response.text)
        return 
    
def get_length_per_date(data:str, date:str):
    """Get the number of rows for a given date

    Parameters
    ----------
    data : str
        Name of the dataset.
    date : str
        Date of the data.

    Returns
    -------
    int
        Number of rows.
    """
    url = f"{URL}{data}" + "/records"
    params = {
        "select": "date",
        "rows": 1,
        "where": f"date='{date}'"
    }
    response = requests.get(url, params=params)
    data=[{}]
    if response.status_code == 200:
        data = response.json()
        return data.get('total_count')
    else:
        print(f"Échec de la requête: {response.status_code}")
        print(response.text)
        return
    

def get_date(data: str, first: bool=True) -> str:
    """Get the minimum or maximum date in a dataset

    Parameters
    ----------
    data : str
        Name of the dataset.
    first : bool, optional
        If True, get the minimum date, else get the maximum date. The default is True.
    
    Returns
    -------
    str
        Date.
    """
    date = "date" if first else "-date"
    
    url = f"{URL}{data}" + "/records"
    params = {
        "select": "date",
        "rows": 1,
        "order_by": date,
    }
    response = requests.get(url, params=params)
    data=[{}]
    if response.status_code == 200:
        data = response.json()
        return data.get('results')[0]['date']
    else:
        print(f"Échec de la requête: {response.status_code}")
        print(response.text)
        return

In [3]:
## jai pas pu push la database sur git car elle est trop lourde

from sqlalchemy import create_engine

engine = create_engine('sqlite:///database.db') 

In [4]:
def preprocess_data(data):
    if 'column_68' in data.columns:
        del data['column_68']

    if 'eolien_terrestre' in data.columns:
        del data['eolien_terrestre']

    if 'eolien_offshore' in data.columns:
        del data['eolien_offshore']

    if 'stockage_batterie' in data.columns:
        del data['stockage_batterie']

    if 'destockage_batterie' in data.columns:
        del data['destockage_batterie']
    return data

def get_data(from_data: str, table_name: str) -> None:
    """Get data from a dataset and insert it into a sqlite database

    Parameters
    ----------
    from_data : str
        Name of the dataset from the API.
    table_name : str
        Name of the table in the database.
    """
    step = 100 # Number of rows to be retrieved per request (API limit)
    start_date = get_date(from_data)
    end_date = get_date(from_data, first = False)
    current_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    
    while current_date <= end_date:
        formatted_date = str(current_date.strftime('%Y-%m-%d'))
        lines_per_date = get_length_per_date(from_data, str(formatted_date))
        print(formatted_date)
        
        for i in range(0, lines_per_date, step):
            rows = min(step, lines_per_date - i)
            try:
                data = fetch_data_by_date(from_data, i, rows, str(formatted_date))
                df = pd.DataFrame(data)
                df = preprocess_data(df)
                df.to_sql(table_name, engine, if_exists='append', index=False)
                    
            except Exception as e:
                print(f"Erreur lors de l'insertion des données pour la date {formatted_date}, offset {i}: {e}")
                print("Réessai dans 5 secondes...")
                time.sleep(5)  
                i -= step  
                continue
                
        current_date += timedelta(days=1)
        

In [ ]:
## RUN TO DOWNLOAD THE DATABASE ##

#print("Filling national table...")
#get_data("eco2mix-national-cons-def", "national")
#get_data("eco2mix-national-tr", "national")

#print("Filling regional table...")
#get_data("eco2mix-regional-cons-def", "regional")
#get_data("eco2mix-regional-tr", "regional")

In [23]:
### FUNCTIONS TO GET DATA FROM DATABASE ###

def get_data_between_dates(table_name, start_date, end_date, db_file='database.db'):
    """
    Get data from a table between two dates using sqlite3.

    Parameters:
        table_name (str): Name of the table in the database.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        db_file (str): Path to the SQLite database file.

    Returns:
        pandas.DataFrame: DataFrame containing the retrieved data.
    """
    conn = sqlite3.connect(db_file)

    query = f"""
        SELECT *
        FROM {table_name}
        WHERE date >= '{start_date}' AND date <= '{end_date}'
    """

    df = pd.read_sql(query, conn)

    conn.close()

    return df

def get_data_by_date(table_name, date, db_file='database.db'):
    """
    Get data from a table for a given date using sqlite3.

    Parameters:
        table_name (str): Name of the table in the database.
        date (str): Date in 'YYYY-MM-DD'
        db_file (str): Path to the SQLite database file.

    Returns:
        pandas.DataFrame: DataFrame containing the retrieved data.
    """
    conn = sqlite3.connect(db_file)

    query = f"SELECT * FROM {table_name} WHERE date = '{date}'"

    df = pd.read_sql(query, conn)

    conn.close()

    return df


def get_trade_data(start_date, end_date,  db_file='database.db'):
    """
    Get trade data between two dates using sqlite3.

    Parameters:
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        db_file (str): Path to the SQLite database file.
    
    Returns:
        pandas.DataFrame: DataFrame containing the retrieved data.
    """

    conn = sqlite3.connect(db_file)
    

    query = f"""
    SELECT date_heure,
           ech_comm_angleterre, 
           ech_comm_espagne, 
           ech_comm_italie, 
           ech_comm_suisse, 
           ech_comm_allemagne_belgique 
    FROM national
    WHERE date_heure >= '{start_date}' AND date_heure <= '{end_date}'
    ORDER BY date_heure;
    """
    
    df = pd.read_sql_query(query, conn)
    conn.close()

    df['date_heure'] = pd.to_datetime(df['date_heure'])
    df.set_index('date_heure', inplace=True)

    return df

def get_aggregated_data_by_filiere(table_name, start_date, end_date, filiere, db_file='database.db'):
    conn = sqlite3.connect(db_file)

    query = f"""
        SELECT libelle_region, SUM({filiere}) as total_production
        FROM {table_name}
        WHERE date >= '{start_date}' AND date <= '{end_date}'
        GROUP BY libelle_region
    """

    df = pd.read_sql(query, conn)
    conn.close()
    return df

<h2 style="padding:16px; color:#FFF; background:#07D">IV. Analyse de la consommation</h2>

La consommation d'électricité est un enjeu majeur tant sur le plan économique que sur celui de l'environnement. Elle évolue au fil du temps en fonction des avancées technologiques et des politiques énergétiques. Ces dernières années, les préoccupations environnementales croissantes ont placé la consommation d'électricité au centre des débats. Nous allons donc essayer de comprendre comment varie la consommation d'électricité en France.

## Évolution de la consommation sur une journée

- un exemple en hiver
- un exemple en été

In [10]:
@interact(day=(1,30))
def line_chart_consumption_november(day: int = 1) -> None:
    """Generate an interactive line chart of electricity consumption in France for a specific day in November 2023.

    Parameters
    ----------
    day : int
        Day of November 2023.
    
    Returns
    -------
    None

    """
    df = get_data_by_date('national', f'2023-11-{day:02}')
    df = df.sort_values(by=['heure'])
    
    plt.figure(figsize=(13.5,3))
    plt.plot(df['heure'], df['consommation'])
    plt.xticks([f'{str(i).zfill(2)}:00' for i in range(24)], [f'{i:02}h' for i in range(24)])
    plt.xlim(left='00:00', right='23:45')
    plt.xlabel('Heure')
    plt.ylabel('Consommation')
    plt.title(f'Consommation électrique en France le {day:02} novembre 2023')

interactive(children=(IntSlider(value=1, description='day', max=30, min=1), Output()), _dom_classes=('widget-i…

## Évolution de la consommation sur une année

- datepicker pour choisir l'année (exclure 2024 pour des raisons évidentes)


In [6]:
start_date_picker = DatePicker(description='Date de début', disabled=False, value=datetime.strptime("2023-11-01", "%Y-%m-%d").date() - timedelta(days=1))
end_date_picker = DatePicker(description='Date de fin', disabled=False, value=datetime.strptime("2023-11-01", "%Y-%m-%d").date())

output = Output()

def draw_figure(start_date, end_date):
    df = get_data_between_dates('national', start_date, end_date)
    df['date_heure'] = pd.to_datetime(df['date_heure'])
    df = df.sort_values(by=['date_heure']).dropna()

    plt.figure(figsize=(10, 5))
    plt.plot(df['date_heure'], df['consommation'], label='Consommation')
    plt.fill_between(df['date_heure'], 0, df['consommation'], alpha=0.2)
    plt.plot(df['date_heure'], df['prevision_j'], label='prevision_j')
    plt.plot(df['date_heure'], df['prevision_j1'], label='prevision_j1')
    plt.xlabel('Date et Heure')
    plt.ylabel('Consommation (MW)')
    plt.title(f'Consommation électrique en France du {start_date} au {end_date}')
    plt.legend(bbox_to_anchor=(1, 1))
    plt.show()

def update_figure(change):
    with output:
        clear_output(wait=True)  # Clear the previous graph
        draw_figure(start_date_picker.value, end_date_picker.value)

start_date_picker.observe(update_figure, names='value')
end_date_picker.observe(update_figure, names='value')

display(VBox([HBox([start_date_picker, end_date_picker]), output]))

update_figure(None)

## Différence de consommation entre les régions

Grille avec :
- Datepicker pour choisir la date/mois/année c'est l'occasion de faire un widget
- Carte de France avec les régions colorées en fonction de leur consommation
- Linechart avec toutes les régions qui se superposent

In [7]:
start_date_picker = DatePicker(description='Date de début', disabled=False, value=datetime.strptime("2023-11-01", "%Y-%m-%d").date() - timedelta(days=1))
end_date_picker = DatePicker(description='Date de fin', disabled=False, value=datetime.strptime("2023-11-01", "%Y-%m-%d").date())

output = Output()

def draw_figure(start_date, end_date):
    df = get_data_between_dates('regional', start_date, end_date)
    df['date_heure'] = pd.to_datetime(df['date_heure'])
    df = df.sort_values(by=['date_heure'])
    plt.figure(figsize=(10, 5))

    for region in df['libelle_region'].unique():
        regional_data = df[df['libelle_region'] == region]
        plt.plot(regional_data['date_heure'], regional_data['consommation'], label=region)
    
    plt.xlabel('Date et Heure')
    plt.ylabel('Consommation (MW)')
    plt.title(f'Consommation électrique par région en France du {start_date} au {end_date}')
    plt.legend(bbox_to_anchor=(1, 1))
    plt.show()

def update_figure(change):
    with output:
        clear_output(wait=True)  # Clear the previous graph
        draw_figure(start_date_picker.value, end_date_picker.value)

start_date_picker.observe(update_figure, names='value')
end_date_picker.observe(update_figure, names='value')

display(VBox([HBox([start_date_picker, end_date_picker]), output]))

update_figure(None)

## Conclusion sur la consommation d'électricité en France

<h2 style="padding:16px; color:#FFF; background:#07D">V. Analyse de la production</h2>

La France est connue pour son industrie nucléaire civil. En effet, le pays a grandement misé sur cette filière pour produire de l'électricité et le nucléaire est la 1ère source de production d'électricité en France. Cependant, le développement des énergies renouvelables est un des grands enjeux de la transition énergétique et la France a également investi dans ces filières. Nous allons donc étudier la production d'électricité en France depuis 2013 et voir l'évolution des différentes filières.

## Production d'électricité en France (hors Corse)

Linechart sur la production d'électricité en France selon plusieurs échelles de temps (jour, semaine, mois, année)

In [8]:
start_date_picker = DatePicker(description='Date de début', disabled=False, value=datetime.strptime("2023-11-01", "%Y-%m-%d").date() - timedelta(days=1))
end_date_picker = DatePicker(description='Date de fin', disabled=False, value=datetime.strptime("2023-11-01", "%Y-%m-%d").date())

output = Output()

def draw_figure(start_date, end_date):
    df = get_data_between_dates('national', start_date, end_date)
    df['date_heure'] = pd.to_datetime(df['date_heure'])
    df = df.sort_values(by=['date_heure']).dropna()

    plt.figure(figsize=(10, 5))
    plt.plot(df['date_heure'], df['nucleaire'], label='Nucléaire')
    plt.plot(df['date_heure'], df['hydraulique'], label='Hydraulique')
    plt.plot(df['date_heure'], df['eolien'], label='Eolien')
    plt.plot(df['date_heure'], df['solaire'], label='Solaire')
    plt.plot(df['date_heure'], df['bioenergies'], label='Bioenergies')
    plt.plot(df['date_heure'], df['gaz'], label='Gaz')
    plt.plot(df['date_heure'], df['fioul'], label='Fioul')
    plt.plot(df['date_heure'], df['charbon'], label='Charbon')

    plt.xlabel('Date et Heure')
    plt.ylabel('Production (MW)')
    plt.title(f'Production électrique en France du {start_date} au {end_date}')
    plt.legend()
    plt.show()

def update_figure(change):
    with output:
        clear_output(wait=True)  # Clear the previous graph
        draw_figure(start_date_picker.value, end_date_picker.value)

start_date_picker.observe(update_figure, names='value')
end_date_picker.observe(update_figure, names='value')

display(VBox([HBox([start_date_picker, end_date_picker]), output]))

update_figure(None)


## Production d'électricité par région et par filière

Grille avec :
- Datepicker pour choisir la date/mois/année
- interact avec choix entre 'régions' et 'filières'
- Piechart sur la somme de la production d'électricité par filière/région sur la période choisie
- Linechart avec toutes les filières/régions qui se superposent sur la période choisie

In [37]:
start_date_picker = DatePicker(description='Date de début', disabled=False, value=datetime.strptime("2023-11-01", "%Y-%m-%d").date() - timedelta(days=1))
end_date_picker = DatePicker(description='Date de fin', disabled=False, value=datetime.strptime("2023-11-01", "%Y-%m-%d").date())

filiere_picker = Dropdown(options=['nucleaire', 'hydraulique', 'eolien', 'solaire', 'bioenergies', 'thermique'], description='Filière', disabled=False)

output = Output()
output_pie_chart = Output()

def draw_figure_all_region(start_date, end_date, filiere):
    df = get_data_between_dates('regional', start_date, end_date)
    df['date_heure'] = pd.to_datetime(df['date_heure'])
    df = df.sort_values(by=['date_heure'])

    regions = df['libelle_region'].unique()

    plt.figure(figsize=(10, 5))
    
    for region in regions:
        regional_data = df[df['libelle_region'] == region]
        plt.plot(regional_data['date_heure'], regional_data[filiere], label=region)

    plt.xlabel('Date et Heure')
    plt.ylabel(f'Production ({filiere}) (MW)')
    plt.title(f'Production électrique {filiere} en France par région du {start_date.strftime("%Y-%m-%d")} au {end_date.strftime("%Y-%m-%d")}')
    plt.legend(bbox_to_anchor=(1, 1))
    plt.show()

def draw_pie_chart(start_date, end_date, filiere):
    df = get_aggregated_data_by_filiere('regional', start_date, end_date, filiere)
    df = df[df['total_production'] > 0]  # filter out regions with no production

    with output_pie_chart:
        clear_output(wait=True)
        plt.figure(figsize=(7, 7))
        plt.pie(df['total_production'], labels=df['libelle_region'], autopct='%1.1f%%', startangle=140)
        plt.title(f'Proportion de production {filiere} par région du {start_date} au {end_date}')
        plt.show()

def update_charts(change):
    with output:
        clear_output(wait=True)
        draw_figure_all_region(start_date_picker.value, end_date_picker.value, filiere_picker.value)
    
    draw_pie_chart(start_date_picker.value, end_date_picker.value, filiere_picker.value)

start_date_picker.observe(update_charts, names='value')
end_date_picker.observe(update_charts, names='value')
filiere_picker.observe(update_charts, names='value')

display(VBox([HBox([start_date_picker, end_date_picker, filiere_picker]), output, output_pie_chart]))

update_charts(None)

## Évolution du la part du renouvelable dans le mix énergétique

Carte de la France avec évolution de la part du renouvelable dans le mix énergétique

## Conclusion sur la production d'électricité en France

<h2 style="padding:16px; color:#FFF; background:#07D">VI. Analyse des échanges commerciaux</h2>

Suite à nos analyses sur la production et la consommation d'électricité en France, nous avons pu voir que la France est un pays qui produit plus d'électricité qu'il n'en consomme. Nous pouvons donc nous dire que la France exporte de l'électricité avec d'autres pays. Nous allons donc étudier les échanges commerciaux d'électricité entre la France et ses pays voisins.

linechart de densité avec les échanges commerciaux entre la France et ses pays voisins avec positif et négatif sur plusieurs échelles de temps (jour, mois, année)

Conclusion à la suite (petite partie)

In [51]:
start_date_picker = DatePicker(description='Date de début', disabled=False, value=datetime.now().date() - timedelta(days=1))
end_date_picker = DatePicker(description='Date de fin', disabled=False, value=datetime.now().date())

output = Output()

def draw_stacked_bar_chart(start_date, end_date):
    df = get_trade_data(start_date, end_date)
    df = df.sort_values(by=['date_heure']).dropna()

    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df_cum = df.clip(lower=0)  # Clip the negative values and sum the values
    df_cum_neg = df.clip(upper=0) # Clip the positive values and sum the values

    colors = ['blue', 'orange', 'green', 'red', 'purple']
    labels = ['Angleterre', 'Espagne', 'Italie', 'Suisse', 'Allemagne/Belgique']
    legend_patches = [plt.Rectangle((0, 0), 1, 1, fc=color) for color in colors]
    
    with output:
        clear_output(wait=True)  
        
        fig, ax = plt.subplots(figsize=(14, 5))
        df_cum.plot(kind='bar', stacked=True, ax=ax, width=0.95, color=colors) # Plot the positive values
        df_cum_neg.plot(kind='bar', stacked=True, ax=ax, width=0.95, color=colors) # Plot the negative values

        ax.axhline(0, color='grey', linewidth=0.8)
        ax.set_ylabel('Valeur des échanges (MW)')
        ax.set_xlabel('Date et Heure')
        ax.set_title('Échanges commerciaux par heure')
        
        ticks = ax.get_xticks()
        ticklabels = [l.get_text() for l in ax.get_xticklabels()]
        ax.set_xticks(ticks[::10]) 
        ax.set_xticklabels(ticklabels[::10], rotation=45, ha="right")

        ax.legend(legend_patches, labels, loc='lower left', bbox_to_anchor=(1, 0))
        plt.tight_layout() 
        plt.show()

def calculate_trade_totals(df):
    exports = df[df < 0].sum().abs()  
    imports = df[df > 0].sum()       

    return exports, imports

def display_trade_totals(start_date, end_date):
    df = get_trade_data(start_date, end_date)
    df = df.sort_values(by=['date_heure']).dropna()

    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    exports, imports = calculate_trade_totals(df)

    totals_df = pd.DataFrame({
        'Exportations': exports,
        'Importations': imports
    })

    with output:
        print("Totaux d'échanges commerciaux (MW):")
        print(totals_df)
        print()

def update_figure(change):
    draw_stacked_bar_chart(start_date_picker.value, end_date_picker.value)
    display_trade_totals(start_date_picker.value, end_date_picker.value)

start_date_picker.observe(update_figure, names='value')
end_date_picker.observe(update_figure, names='value')

display(VBox([HBox([start_date_picker, end_date_picker]), output]))

update_figure(None)


<h2 style="padding:16px; color:#FFF; background:#07D">Conclusion de l'étude</h2>